<a href="https://colab.research.google.com/github/Polar-Tsai/Lesson_Record/blob/main/house_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train_df = pd.read_csv("https://raw.githubusercontent.com/Elwing-Chou/ml0223/main/house/train.csv", encoding="utf-8")
test_df = pd.read_csv("https://raw.githubusercontent.com/Elwing-Chou/ml0223/main/house/test.csv", encoding="utf-8")
train_df

In [ ]:
# 合併train、test

datas = pd.concat([train_df, test_df], axis=0)
datas = datas.drop(["SalePrice"], axis=1).reset_index(drop=True)
datas

In [ ]:
import numpy as np
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
y = train_df["SalePrice"]
print("skew:", skew(y))
print("lambda:", boxcox_normmax(1 + y))
sns.distplot(y)

In [ ]:
# 調整後變常態分佈 with log1p
y = train_df["SalePrice"]
y_train_log1p = np.log1p(y)
print("skew:", skew(y_train_log1p))
sns.distplot(y_train_log1p)

In [ ]:
# 調整後變常態分佈 with boxcox1p
y = train_df["SalePrice"]
lamda = boxcox_normmax(y)
y_train_boxcox1p = boxcox1p(y, lamda)
print("skew:", skew(y_train_log1p))
sns.distplot(y_train_log1p)

In [ ]:
# 檢查缺失值
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

In [ ]:
# 去除少一半資料的欄位
datas_drop = datas.drop(["Id", "PoolQC", "MiscFeature", "Alley", "Fence"], axis=1)

In [ ]:
idx = datas_drop.dtypes != "object"
number_idx = datas_drop.dtypes[idx].index.drop(["MSSubClass"])
number_idx

In [ ]:
import pandas as pd
datas_drop = pd.get_dummies(datas_drop)
datas_drop = pd.get_dummies(datas_drop, columns=["MSSubClass"])
datas_drop

In [ ]:
# 調整偏度
def getskew(series):
    return skew(series.dropna() + 1)
skewness = (datas_drop[number_idx].apply(getskew)
                  .sort_values(ascending=False))
need_saved = skewness[skewness > 1].index
print(need_saved)
skewness[skewness > 1]

In [ ]:
def save(s):
    lamda = boxcox_normmax(s.dropna() + 1)
    return boxcox1p(s, lamda)
datas_drop[need_saved] = datas_drop[need_saved].apply(save)
skewness = (datas_drop[number_idx].apply(getskew)
                  .sort_values(ascending=False))
skewness

In [ ]:
# 補上缺失值
datas_drop = datas_drop.fillna(datas_drop.median())
datas_drop

## Scaling
1. MinMax: 最小值=0；最大值=1
> 疑慮: 如果離群值和大家距離差太多，大家的值會因為她而導致都擠在一起
2. Standard: (x-μ)/σ 使平均=0；標準差=1
3. Robost Scaler: (x-Q1)/(Q3-Q1)
>* MinMax改良版
>* 去除離群值後，計算間距

### 哪個好??都試過才知道

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
datas_norm = pd.DataFrame(scaler.fit_transform(datas_drop),
              columns
              =datas_drop.columns)
x_train = datas_norm.iloc[:len(train_df)]

x_predict = datas_norm.iloc[len(train_df):]
# y_train_log1p

In [ ]:
# x_train: 1460 * 300
# x_predict: 1459 * 300
# y_train_log1p: 1460

#### 分類類別問題，以非線性函數的表現較佳
#### 迴歸類別問題，則以 **線性函數**較好
原因:
1. 過擬和(些微震盪也放入的話，萬一那些資料是壞資料，反而會導致資料表現差)
2. 隨機森林沒有更細微劃分(分支後的結果並沒有做到更好的分類)

Skewness偏度:
> <0: 左偏；=0: 常態；>0: 右偏

---

### 線性迴歸本身是不需要做scaling
y = a1x1 + a2x3 + a3x3 + b
### 但在使用梯度下降下，依然需要做scaling。與此同時，要加上懲罰項
Loss(MSE) = (Σ(pre-正確)^2)/n

懲罰項
1. Lasso Regression(L1正則化)
>* 只考慮真正影響非常大的欄位，其他慢慢刪減至無
>* **有特徵挑選的效果**
>* 好用在特徵過多的資料
>>* α(|a1| + |a2| + |a3|)
>>* 如果a1下降1，Loss上升5
>>* 如果a2下降5，Loss上升1-> Loss變動幅度小，a2設為=0
2. Ridge Regression(L2正則化: 嶺迴歸)
>* 減少依賴過大係數
>* **使係數平衡**
>>* α(a1^2 + a2^2 + a3^2)

3. ElasticNet Regression
>* 結合Lasso+Ridge


### 哪個好??都試過才知道

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet

# Lasso
cv = LassoCV(cv=10)
cv.fit(x_train, y_train_log1p)
print(cv.alpha_)
model = Lasso(cv.alpha_)
model.fit(x_train, y_train_log1p)
pre = model.predict(x_predict)
result = pd.DataFrame({
    "Id":test_df["Id"],
    "SalePrice":np.expm1(pre) # exprenatio - 1
})
result.to_csv("lasso.csv", encoding="utf-8", index=False)
result


In [ ]:
# Ridge
cv = RidgeCV(cv=10)
cv.fit(x_train, y_train_log1p)
print(cv.alpha_)
model = Ridge(cv.alpha_)
model.fit(x_train, y_train_log1p)
pre = model.predict(x_predict)
result = pd.DataFrame({
    "Id":test_df["Id"],
    "SalePrice":np.expm1(pre)
})
result.to_csv("ridge.csv", encoding="utf-8", index=False)
result

In [ ]:
# Elastic
cv = ElasticNetCV(cv=10)
cv.fit(x_train, y_train_log1p)
print(cv.alpha_)
model = ElasticNet(cv.alpha_)
model.fit(x_train, y_train_log1p)
pre = model.predict(x_predict)
result = pd.DataFrame({
    "Id":test_df["Id"],
    "SalePrice":np.expm1(pre)
})
result.to_csv("elastic.csv", encoding="utf-8", index=False)
result

為了讓資料能進行分析，必須先讓他們變成常態分佈(個別使用log1p / boxcox1p)

取得結果後，要記得再換回原本的y值(expm1 / inv_boxcox1p)
1. log1p > expm1
2. boxcox1p > inv_boxcox1p